In [ ]:
%matplotlib inline

from model import BaryonModel
from data import Data
from IPython import display

from torch import nn

import torch

In [ ]:
batch_size = 64
ngpu = 0
# Epochs set to '1' for testing
epochs = 1500
learning_rate = 1e-4
beta1 = 0.9
n_samples = 8192

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
dataset = Data('NetbaryonDis_OSG3DAuAu19.6_tune18.3_wBulk_22momdeltaf', n_samples )

data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
model = BaryonModel()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [ ]:
loss_func = nn.MSELoss()

In [ ]:
from hydroml import plot

loss_total = []

for epoch in range(epochs):
    for i, data in enumerate(data_loader):
        baryons = data[0]
        protons = data[1]

        optimizer.zero_grad()

        output = model(baryons)

        loss = loss_func(output, protons)

        loss.backward()
        optimizer.step()

        loss_total.append(loss.detach().numpy())

        if i % 128 == 0:
            display.clear_output(wait=True)

            # Uncommenting this will plot the loss and the generated output vs the actual output of the training sample.
            #plot.plot_telemetry(loss_total, output.detach().numpy(), protons)

            print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(dataset) // batch_size}], loss: {loss:.4f}")

In [ ]:
torch.save(model, './baryon_model_19gev.pt')